# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="assets/pic1.png"/>

Then you will want to run the above command:

<img src="assets/pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

## Data Exploration

In [1]:
!python -m pip install black isort

     |████████████████████████████████| 1.4 MB 22.6 MB/s            


In [2]:
import sys
sys.version

'3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) \n[GCC 9.3.0]'

In [3]:
%load_ext autoreload

import pathlib
import pandas as pd
import numpy as np
import math
import json

%matplotlib inline

# read in the json files
data_dir = pathlib.Path('data')
files = {stem: data_dir / f"{stem}.json" 
         for stem in ('portfolio','profile', 'transcript')}
dataframes = {key: pd.read_json(file, orient='records', lines=True)
              for key, file in files.items()}
portfolio = dataframes.get("portfolio")
profile = dataframes.get("profile")
transcript = dataframes.get("transcript")

## Data Exploration
### Review DataFrames

In [4]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


### Review Transcript specifics

## Data Cleaning

The following cell will write a file into a local module and be loaded to clean
the dataframes before feature engineering will be performed. This will add
values to null that were coded as another value, expand nested datatypes into
columns; such as lists; categorize columns where appropriate, and convert
date columns into DateTime values.

In [7]:
module_dir = pathlib.Path('capstone_tools')
module_dir.mkdir(exist_ok=True)
(module_dir / "__init__.py").touch(exist_ok=True)

In [8]:
%%writefile capstone_tools/data_cleaners.py
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass
from itertools import chain
from abc import ABC

import pandas as pd
import numpy as np


def assign_empty(ser: pd.Series, assign_to: Dict[Any, Any]) -> pd.Series:
    """
    Convert a value in a series to a new value - intended for assigning nan to
    coded values
    """
    ser = ser.copy()
    for key, val in assign_to.items():
        ser[ser == key] = val
    return ser


def assign_categories(
    df: pd.DataFrame, category_cols: List[Any]
) -> pd.DataFrame:
    """
    Assign columns as categories specified in category_cols
    
    Parameters
    ----------
    df : DataFrame
        dataframe to assign new columns
    category_cols : list[str]
        list of column names to assign as categories
    is_ordered : bool
        sets columns as an ordered category
    """
    new_df = df.copy()
    for col in category_cols:
        new_df = new_df.assign(**{col: (lambda df: pd.Categorical(df[col]))})

    return new_df

def assign_string_cols(
    df: pd.DataFrame, string_cols: List[Any]
) -> pd.DataFrame:
    """
    Assign columns as categories specified in category_cols
    
    Parameters
    ----------
    df : DataFrame
        dataframe to assign new columns
    category_cols : list[str]
        list of column names to assign as categories
    is_ordered : bool
        sets columns as an ordered category
    """
    # Note: assigning is broken if you try to pass multiple evaluation values 
    new_df = df.copy()
    for col in string_cols:
        new_df = new_df.assign(
            **{col: (lambda df_: df_[col].astype("string"))}
        )

    return new_df


@dataclass
class CleanerBase(ABC):
    """ABC for Cleaners"""
    df: pd.DataFrame

    def clean(self) -> pd.DataFrame:
        raise NotImplementedError("Implement clean method")
    
    @classmethod
    def register(cls, key: str = None) -> None:
        """Register the Cleaner"""
        global _registered_cleaners
        if not key:
            key = cls.__name__
        _registered_cleaners[key] = cls


class RegistrationError(KeyError):
    """Registration Error for Cleaner Classes"""


# dict[str, CleanerBase]
_registered_cleaners = {}
    

def clean(df: pd.DataFrame, key: str) -> pd.DataFrame:
    """Clean a dataframe provided a key to a registered CleanerBase class"""
    global _registered_cleaners
    cleaner = _registered_cleaners.get(key)
    
    if not cleaner:
        raise RegistrationError(
            f"Cleaner Class not registered, check key: {key}"
        )
    return cleaner(df).clean()


class EventLogCleaner(CleanerBase):
    """Cleaner for Transcripts / Event Log DataFrames"""
    def clean(self) -> pd.DataFrame:
        """
        Clean the Event Log / Transcript DataFrame
        """
        STR_COLS = ("offer_id", "person")
        CAT_COLS = ("event", )
        for col in  CAT_COLS:
            assert col in self.df.columns, f"Required column missing: {col}"
            
        return (
            self.df
            .pipe(self._expand_value_col)
            .pipe(lambda df_: assign_categories(df_, CAT_COLS))
            .pipe(lambda df_: assign_string_cols(df_, STR_COLS))
            .drop("value", axis=1)
        )
    
    @classmethod
    def _expand_value_col(cls, df: pd.DataFrame) -> pd.DataFrame:
        """
        Add expand categories within the "value" column within an Event Log 
        DataFrame.

        ~~Pipeable~~
        """
        value_map = cls._get_value_map(df) # Performs Validation
        for event, subdf in df.groupby("event"):
            idx = subdf.index
            cols = value_map.get(event, ())

            for col in cols:
                clean_col = col.strip().replace(' ', '_')
                new_col_vals = subdf["value"].apply(lambda x: x[col])
                if clean_col not in df.columns:
                    df = df.assign(**{clean_col: np.nan})
                df.loc[idx, clean_col] = new_col_vals
        return df
    
    @staticmethod
    def _get_value_map(df: pd.DataFrame) -> Dict[str, Tuple[str]]:
        """
        Generate a dictionary a key map to contents in the "value" column
        depending on event type from "event" column

        Parameters
        ----------
        df : DataFrame containing columns "event" and "value" (event log)
            Event Log DataFrame

        Returns
        -------
        dict[str, tuple[str, ...]]
            Map of event categories containing keys within the "value" column 
        """
        REQ_COLS = ("event", "value")
        # Validation
        for col in REQ_COLS:
            assert col in df.columns, f"DataFrame needs: {REQ_COLS!r}"
            
        value_map = {}
        for event_type, subdf in df.groupby("event"):
            value_map[event_type] = tuple(subdf["value"].iloc[0].keys())
        return value_map


class PortfolioCleaner(CleanerBase):
    """Cleaner for Portfollio DataFrames"""
    def clean(self) -> pd.DataFrame:
        """Clean Portfolio DataFrame"""
        STR_COLS = ("id", )
        CAT_COLS = ("offer_type", )
        
        for col in chain(STR_COLS, CAT_COLS):
            assert col in self.df.columns, f"Required column missing: {col}"
            
        return (
            self.df
            .pipe(lambda df_: assign_categories(df_, CAT_COLS))
            .pipe(lambda df_: assign_string_cols(df_, STR_COLS))
            .pipe(self.make_one_hot_from_channel)
        )
    
    @staticmethod
    def make_one_hot_from_channel(df: pd.DataFrame) -> pd.DataFrame:
        """
        Converts the channels column into a series of one-hot columns for each
        category for the portfolio dataframe
        """
        assert "channels" in df.columns, f"DataFrame needs 'channels' column"
        
        df = df.copy()
        # Note: assumes one list has all categories - this is true by 
        #       inspection, but not generally true
        idx = df.channels.str.len().argmax() 
        max_list = tuple(df.loc[idx, "channels"])
        new_df = df.copy()
        for cat in max_list:
            new_df = new_df.assign(
                **{
                    cat: (
                        lambda df_: (
                            df_.channels.transform(lambda x: cat in x)
                        ).astype(int)
                    )
                }
            )
        return new_df.drop('channels', axis=1)


class ProfileCleaner(CleanerBase):
    """Cleaner for Profile DataFrame"""
    def clean(self) -> pd.DataFrame:
        """Clean Profile DataFrame"""
        STR_COLS = ("id", )
        CAT_COLS = ("gender", )
        DATE_COLS = ("became_member_on", )
        NAN_INPUTS = {"age": {118: np.nan}}
        for col in chain(STR_COLS, CAT_COLS, DATE_COLS, NAN_INPUTS.keys()):
            assert col in self.df.columns, f"Required column missing: {col}"
            
        return (
            self.df
            .pipe(lambda df_: assign_categories(df_, CAT_COLS))
            .pipe(lambda df_: assign_string_cols(df_, STR_COLS))
            .assign(**{
                col: (lambda df_: pd.to_datetime(
                    df_['became_member_on'], format='%Y%m%d'
                )) for col in DATE_COLS
            })
            .assign(**{
                col: (lambda df_: assign_empty(df_[col], na_map))
                for col, na_map in NAN_INPUTS.items()
            })
        )

def __register_classes():
    """Register Base Cleaners"""
    klass_registers = {
        'transcript': EventLogCleaner,
        'portfolio': PortfolioCleaner,
        'profile': ProfileCleaner
    }
    for key, klass in klass_registers.items():
        klass.register(key)

__register_classes()


Overwriting capstone_tools/data_cleaners.py


In [9]:
!python -m isort capstone_tools/data_cleaners.py
!python -m black --line-length=79 capstone_tools/data_cleaners.py

Fixing /home/ec2-user/SageMaker/udacity-mle-capstone-project/capstone_tools/data_cleaners.py
reformatted capstone_tools/data_cleaners.py

All done! ✨ 🍰 ✨
1 file reformatted.


In [10]:
%autoreload 2
from capstone_tools.data_cleaners import clean

In [11]:
from IPython.display import display

In [12]:
clean_dataframes = {key: clean(df, key) for key, df in dataframes.items()}

In [13]:
portfolio = clean_dataframes['portfolio']
profile = clean_dataframes['profile']
transcript = clean_dataframes['transcript']

In [14]:
print('Portfolio:')
display(portfolio.head())
print('Profile:')
display(profile.head())
print('Transcript:')
display(transcript.head())

Portfolio:


,reward,difficulty,duration,offer_type,id,web,email,mobile,social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0


Profile:


,gender,age,id,became_member_on,income
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


Transcript:


,person,event,time,offer_id,reward,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


## Data Visualization

In [15]:
profile

,gender,age,id,became_member_on,income
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN
...,...,...,...,...,...
16995,F,45.0,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0
16996,M,61.0,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0
16997,M,49.0,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0
16998,F,83.0,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0


## Feature Generation

In [16]:
transcript.event.unique()

['offer received', 'offer viewed', 'transaction', 'offer completed']
Categories (4, object): ['offer received', 'offer viewed', 'transaction', 'offer completed']

In [17]:
completed_offers = transcript.query("event == 'offer completed'")


In [18]:
first_id = completed_offers.iloc[0]

In [19]:
first_id

person      9fa9ae8f57894cc9a3b8a9bbe0fc1b2f
event                        offer completed
time                                       0
offer_id    2906b810c7d4411798c6938adc9daaa5
reward                                     2
amount                                   NaN
Name: 12658, dtype: object

In [28]:
first_id.person

'9fa9ae8f57894cc9a3b8a9bbe0fc1b2f'

In [21]:
transcript.query("event == 'offer received'").query(f"person == '{first_id.person}' and offer_id == '{first_id.offer_id}'")

,person,event,time,offer_id,reward,amount
36,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN


In [22]:
transcript.query("event == 'offer viewed'").query(f"person == '{first_id.person}' and offer_id == '{first_id.offer_id}'")

,person,event,time,offer_id,reward,amount
12656,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer viewed,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN


In [23]:
transcript.query("event == 'transaction'").query(f"person == '{first_id.person}'")

,person,event,time,offer_id,reward,amount
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,<NA>,NaN,34.56
27850,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,42,<NA>,NaN,21.55
44327,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,114,<NA>,NaN,32.87
51350,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,156,<NA>,NaN,19.99
65854,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,168,<NA>,NaN,20.97
81403,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,204,<NA>,NaN,31.09
132814,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,354,<NA>,NaN,18.42
145382,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,390,<NA>,NaN,29.71
147220,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,396,<NA>,NaN,32.53
148933,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,402,<NA>,NaN,33.36


In [29]:
len(transcript.person.unique())

17000

,person,event,time,offer_id,reward,amount,duration
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,7.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,7.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,10.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,5.0
...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,<NA>,NaN,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,<NA>,NaN,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,<NA>,NaN,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,<NA>,NaN,3.53,NaN


In [30]:
transcript.query(f"person == '{first_id.person}'")

,person,event,time,offer_id,reward,amount
36,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
12656,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer viewed,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,<NA>,NaN,34.56
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2.0,NaN
27850,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,42,<NA>,NaN,21.55
44327,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,114,<NA>,NaN,32.87
51350,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,156,<NA>,NaN,19.99
65854,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,168,<NA>,NaN,20.97
81403,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,204,<NA>,NaN,31.09
110866,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,336,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN


In [102]:
# valid values - within the duration of the offer
# ack values - user observed the offer
df = (
    pd.merge(transcript, portfolio[['id', 'duration']],
             how='left', left_on='offer_id', right_on='id',
             suffixes=('', '_offer'))
    .drop('id', axis=1)
    .rename(columns={'duration': 'valid_duration'})
    .assign(valid_start=np.nan,
            time_elapsed=np.nan)
 )
valid_starts = df.query("event == 'offer received'")['time']
df.loc[valid_starts.index, 'valid_start'] = valid_starts.values
df = (
    df.assign(
        valid_start=df.valid_start.ffill(),
        valid_duration=lambda df_: df_.valid_duration.ffill(),   
    )
    .assign(time_elapsed=lambda df_: df_.time - df_.valid_start)
    .assign(is_valid=lambda df_: df_.time_elapsed <= df_.valid_duration)
#     .assign(ack=np.nan)
)
ack_starts = df.query("event == 'offer viewed'").index
# df.loc[ack_starts, 'acknowledged'] = True
# df = (
#     df
#     .assign(ack=lambda df_: df.ack.ffill())
#     .assign(ack=lambda df_: df.ack.bfill(False))
#     .assign(offer_success=lambda df_: df.ack & df.is_valid)
#  )

In [104]:
df

,person,event,time,offer_id,reward,amount,valid_duration,valid_start,time_elapsed,is_valid
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,7.0,0.0,0.0,True
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0.0,True
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,7.0,0.0,0.0,True
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,10.0,0.0,0.0,True
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,5.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,<NA>,NaN,1.59,10.0,576.0,138.0,False
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,<NA>,NaN,9.53,10.0,576.0,138.0,False
306531,a00058cf10334a308c68e7631c529907,transaction,714,<NA>,NaN,3.61,10.0,576.0,138.0,False
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,<NA>,NaN,3.53,10.0,576.0,138.0,False


In [24]:
transcript.query("event == 'offer completed'").query(f"person == '{first_id.person}' and offer_id == '{first_id.offer_id}'")

,person,event,time,offer_id,reward,amount
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2.0,NaN


In [25]:
first_id.offer_id

'2906b810c7d4411798c6938adc9daaa5'

In [26]:
portfolio.query(f"id == '{first_id.offer_id}'")

,reward,difficulty,duration,offer_type,id,web,email,mobile,social
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0


In [27]:
transcript.query(f"person == '{first_id.person}'")

,person,event,time,offer_id,reward,amount
36,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
12656,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer viewed,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,<NA>,NaN,34.56
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2.0,NaN
27850,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,42,<NA>,NaN,21.55
44327,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,114,<NA>,NaN,32.87
51350,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,156,<NA>,NaN,19.99
65854,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,168,<NA>,NaN,20.97
81403,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,204,<NA>,NaN,31.09
110866,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,336,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN


## Removing Bad Values